[ver código](https://github.com/python-visualization/folium/blob/master/folium/plugins/heat_map_withtime.py)

### Objetivo

Criar uma mapa de calor do número de mortes causadas pela COVID-19 entre os dias 25-03-2020 e 11-04-2020 no estado de São Paulo.

### Origem dos Dados

Os dados foram extraídos dos boletins epidemiológicos que estão disponíveis na página [SITUAÇÃO EPIDEMIOLÓGICA DO ESTADO DE SÃO PAULO.](http://www.saude.sp.gov.br/cve-centro-de-vigilancia-epidemiologica-prof.-alexandre-vranjac/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus-covid-19/situacao-epidemiologica) do site da prefeitura da cidade de São Paulo. Caso a página não esteja disponível, você poderá encontrar os pdfs dos boletins em questão clicando [aqui](https://github.com/SClovesgtx/SClovesgtx/tree/master/boletins).

Os dados parseados dos boletins podem ser obtidos em csv clicando [aqui](https://www.kaggle.com/clovesgtx/covid19-municpios-de-so-paulo).

In [7]:
import pandas as pd
import folium
from folium import plugins
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [3]:
df = pd.read_csv("../data/covid-19-municipios-sp.csv")

In [9]:
df.Dia =  pd.to_datetime(df.Date, format='%d-%m-%Y')

In [10]:
df.head()

,Date,Bulletin,City,Confirmed,Deaths
0,17-04-2020,51,ADAMANTINA,1.0,0.0
1,17-04-2020,51,GUARUJÁ,27.0,2.0
2,17-04-2020,51,PILAR DO SUL,3.0,0.0
3,17-04-2020,51,ÁGUAS DE LINDÓIA,1.0,1.0
4,17-04-2020,51,GUARULHOS,295.0,28.0


### Obtendo geocoordenadas

In [11]:
def do_geocode(address):
    geolocator = Nominatim(user_agent="clovesgtx@hotmail.com")
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [21]:
dic = dict()
for municipio in df.City.unique():
    if municipio not in dic.keys():
        m = str(municipio) + " - SP"
        location = do_geocode(m)
        if location:
            dic[municipio] = {"lat": location.latitude,
                             "lon": location.longitude}
        else:
            dic[municipio] = {"lat": None,
                             "lon": None}

In [23]:
df["Latitude"] = [dic[m]["lat"] for m in df.City]
df["Longitude"] = [dic[m]["lon"] for m in df.City]

In [24]:
df.head()

,Date,Bulletin,City,Confirmed,Deaths,Latitude,Longitude
0,17-04-2020,51,ADAMANTINA,1.0,0.0,-21.686652,-51.076298
1,17-04-2020,51,GUARUJÁ,27.0,2.0,-23.992777,-46.255833
2,17-04-2020,51,PILAR DO SUL,3.0,0.0,-23.812168,-47.720186
3,17-04-2020,51,ÁGUAS DE LINDÓIA,1.0,1.0,-22.471885,-46.629103
4,17-04-2020,51,GUARULHOS,295.0,28.0,-23.443060,-46.524459


## Mapa Base

In [25]:
def generateBaseMap(default_location=[-22.292690, -48.558171], default_zoom_start=7):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [26]:
base_map = generateBaseMap()
base_map

# Mapa de Calor

In [31]:
from folium.plugins import HeatMap

map_confirmados = generateBaseMap()
HeatMap(data=df[['Latitude', 'Longitude', 'Deaths']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_confirmados)
map_confirmados

In [32]:
map_confirmados.save("../maps/heat-map-of-deaths-by-covid-in-sao-paulo-br.html")

# Adicionando Controlador de Tempo

In [29]:
df_day_list = []
for day in df.Dia.sort_values().unique():
    df_day_list.append(df.loc[df.Dia == day, ['Latitude', 'Longitude', 'Deaths']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

{heatmapOptions: {
                        radius: 13,
                        minOpacity: 0.05,
                        maxOpacity: 0.6,
                        scaleRadius: false,
                        useLocalExtrema: false,
                        defaultWeight: 1,
                        gradient: {0.4: 'blue', 0.65: 'lime', 1: 'red'}
                    }

In [33]:
from folium.plugins import HeatMapWithTime

mortes_confirmadas_map = generateBaseMap()
casos_covid_time = HeatMapWithTime(data=df_day_list, max_opacity=0.6,radius=13)
casos_covid_time.add_to(mortes_confirmadas_map)
mortes_confirmadas_map

In [35]:
mortes_confirmadas_map.save("../maps/heat-map-of-deaths-by-covid-in-sao-paulo-br-with-time.html")